In [ ]:
# Required installations (for Google Colab)
!apt-get install -y tesseract-ocr
!pip install pytesseract opencv-python numpy

import cv2
import pytesseract
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from PIL import Image
import io

# Configure Tesseract path (for Colab)
pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"


def preprocess_image(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(blur, 50, 200)
    return edged

def find_license_plate_contours(edged, img):
    contours, _ = cv2.findContours(edged, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:15]

    for contour in contours:
        perimeter = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.018 * perimeter, True)

        if len(approx) == 4:
            x, y, w, h = cv2.boundingRect(approx)
            if w > 80 and h > 20:
                license_plate = img[y:y + h, x:x + w]
                return license_plate, approx
    return None, None

def enhance_plate_image(plate_img):
    plate_gray = cv2.cvtColor(plate_img, cv2.COLOR_BGR2GRAY)
    plate_gray = cv2.bilateralFilter(plate_gray, 11, 17, 17)
    _, plate_thresh = cv2.threshold(plate_gray, 150, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return plate_thresh

def ocr_license_plate(plate_img):
    processed_img = enhance_plate_image(plate_img)
    text = pytesseract.image_to_string(processed_img, config='--psm 8')
    return text.strip()


uploaded_files = files.upload()
for file_name, file_data in uploaded_files.items():

    image = Image.open(io.BytesIO(file_data)).convert("RGB")
    img = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)


    edged = preprocess_image(img)
    license_plate, approx = find_license_plate_contours(edged, img)

    if license_plate is not None:
        cv2.drawContours(img, [approx], -1, (0, 255, 0), 3)
        plate_text = ocr_license_plate(license_plate)
        print(f"Detected License Plate Number from {file_name}: {plate_text}")

        # Display result
        plt.figure(figsize=(6,3))
        plt.imshow(cv2.cvtColor(license_plate, cv2.COLOR_BGR2RGB))
        plt.title(f"License Plate: {plate_text}")
        plt.axis('off')
        plt.show()
    else:
        print(f"No license plate detected in {file_name}")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
